# 📊 Exploración de Datos - DeAcero Steel Price Predictor

Este notebook explora los datos recopilados de múltiples fuentes para la predicción de precios de varilla corrugada.

## Objetivos:
1. Cargar todas las series temporales desde archivos raw
2. Analizar la calidad y completitud de los datos
3. Identificar patrones y tendencias iniciales
4. Evaluar correlaciones preliminares
5. Preparar datos para análisis de features

## Fuentes de Datos (10 fuentes funcionales):

### 📈 DATOS DIARIOS (críticos para predicción)
- **Yahoo Finance**: Commodities, índices y acciones (12 series)
- **Raw Materials**: Mineral de hierro y carbón de coque - proxies vía acciones mineras (10 series)
- **Banxico**: Tipo de cambio, TIIE, UDIS (8 series)
- **FRED**: Indicadores económicos US (8 series)
- **LME**: Precios de metales (10 series)
- **investing**: Precio de la varilla

### 📊 DATOS MENSUALES (importantes para contexto)
- **World Bank Monthly** ⭐: Commodities mensuales Pink Sheet - Mineral de hierro, carbón, metales (9 series)
- **INEGI**: Indicadores económicos México (16 series)
- **AHMSA**: Precios históricos acero México (7 series)

### 📉 DATOS ANUALES/LIMITADOS
- **World Bank**: Indicadores macroeconómicos anuales (8 series)
- **Trading Economics**: Solo valores actuales (5 series)


In [ ]:
# Configuración inicial
import sys
import os
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
import glob
import json
from datetime import datetime, timedelta
from typing import Dict, List, Tuple

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 100)

print("📚 Librerías importadas exitosamente")
print(f"📅 Fecha de análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# Función para cargar datos raw
def load_raw_data(raw_dir: str = '../data/raw') -> Dict[str, pd.DataFrame]:
    """
    Carga todos los archivos CSV del directorio raw
    
    Returns:
        Dict con DataFrames por fuente
    """
    all_data = {}
    
    # Obtener todos los archivos CSV
    csv_files = glob.glob(os.path.join(raw_dir, '*.csv'))
    
    print(f"📁 Encontrados {len(csv_files)} archivos CSV en {raw_dir}")
    
    # Agrupar por fuente
    sources = {}
    for file in csv_files:
        filename = os.path.basename(file)
        source = filename.split('_')[0]
        
        if source not in sources:
            sources[source] = []
        sources[source].append(file)
    
    # Cargar datos por fuente
    for source, files in sources.items():
        print(f"\n📊 Cargando {len(files)} archivos de {source}...")
        source_data = {}
        
        for file in files:
            try:
                # Cargar CSV
                df = pd.read_csv(file)
                
                # Obtener nombre de la serie
                filename = os.path.basename(file)
                parts = filename.replace('.csv', '').split('_')
                
                # El nombre de la serie es todo menos la fuente y la fecha
                series_name = '_'.join(parts[1:-1])
                
                # Convertir fecha a datetime si existe
                # Manejar diferentes nombres de columna de fecha
                if 'fecha' in df.columns:
                    df['fecha'] = pd.to_datetime(df['fecha'])
                    df = df.sort_values('fecha')
                elif 'Date' in df.columns:
                    # Para archivos FRED que usan 'Date' en lugar de 'fecha'
                    df['fecha'] = pd.to_datetime(df['Date'])
                    df = df.drop(columns=['Date'])
                    df = df.sort_values('fecha')
                    
                    # Identificar columna de valor para FRED
                    # La segunda columna suele ser el valor
                    value_cols = [col for col in df.columns if col != 'fecha']
                    if value_cols and 'valor' not in df.columns:
                        df['valor'] = df[value_cols[0]]
                
                source_data[series_name] = df
                
                # Cargar metadata si existe
                metadata_file = file.replace('.csv', '_metadata.json')
                if os.path.exists(metadata_file):
                    with open(metadata_file, 'r') as f:
                        metadata = json.load(f)
                        source_data[f"{series_name}_metadata"] = metadata
                        
                print(f"   ✅ {series_name}: {len(df)} registros")
                
            except Exception as e:
                print(f"   ❌ Error cargando {os.path.basename(file)}: {str(e)}")
        
        all_data[source] = source_data
    
    return all_data

print("🔧 Función de carga de datos definida")


## 📂 Carga de Datos Raw

Cargamos todas las series temporales desde los archivos CSV guardados en `data/raw/`.


In [ ]:
# Cargar todos los datos raw
all_data = load_raw_data()

# Resumen de datos cargados
print("\n" + "="*60)
print("📊 RESUMEN DE DATOS CARGADOS")
print("="*60)

total_series = 0
total_points = 0

for source, data in all_data.items():
    # Filtrar solo DataFrames (no metadata)
    dfs = {k: v for k, v in data.items() if isinstance(v, pd.DataFrame)}
    
    series_count = len(dfs)
    points_count = sum(len(df) for df in dfs.values())
    
    total_series += series_count
    total_points += points_count
    
    print(f"\n📁 {source}:")
    print(f"   Series: {series_count}")
    print(f"   Puntos totales: {points_count:,}")
    
    # Mostrar rango de fechas si existe
    for name, df in dfs.items():
        if 'fecha' in df.columns and not df.empty:
            fecha_min = df['fecha'].min()
            fecha_max = df['fecha'].max()
            print(f"   • {name}: {fecha_min.strftime('%Y-%m-%d')} a {fecha_max.strftime('%Y-%m-%d')} ({len(df):,} puntos)")

print("\n" + "="*60)
print(f"📈 TOTAL GENERAL:")
print(f"   Series temporales: {total_series}")
print(f"   Puntos de datos: {total_points:,}")
print("="*60)


## 📊 Análisis de Completitud de Datos

Analizamos la completitud y calidad de cada serie temporal.


In [ ]:
# Análisis de completitud de datos
def analyze_data_quality(all_data: Dict) -> pd.DataFrame:
    """Analiza la calidad y completitud de los datos"""
    
    quality_report = []
    
    for source, data in all_data.items():
        # Filtrar solo DataFrames
        dfs = {k: v for k, v in data.items() if isinstance(v, pd.DataFrame)}
        
        for name, df in dfs.items():
            report = {
                'fuente': source,
                'serie': name,
                'registros': len(df),
                'columnas': len(df.columns),
                'valores_nulos': df.isnull().sum().sum(),
                'pct_nulos': (df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100) if len(df) > 0 else 0
            }
            
            # Análisis de fechas si existe columna fecha
            if 'fecha' in df.columns and not df.empty:
                report['fecha_inicio'] = df['fecha'].min()
                report['fecha_fin'] = df['fecha'].max()
                report['dias_totales'] = (df['fecha'].max() - df['fecha'].min()).days
                
                # Detectar frecuencia
                if len(df) > 1:
                    date_diffs = df['fecha'].diff().dropna()
                    mode_diff = date_diffs.mode()[0] if not date_diffs.empty else pd.Timedelta(days=1)
                    
                    if mode_diff.days == 1:
                        report['frecuencia'] = 'Diaria'
                    elif mode_diff.days == 7:
                        report['frecuencia'] = 'Semanal'
                    elif 28 <= mode_diff.days <= 31:
                        report['frecuencia'] = 'Mensual'
                    elif 90 <= mode_diff.days <= 92:
                        report['frecuencia'] = 'Trimestral'
                    elif 365 <= mode_diff.days <= 366:
                        report['frecuencia'] = 'Anual'
                    else:
                        report['frecuencia'] = 'Irregular'
                else:
                    report['frecuencia'] = 'N/A'
            
            # Análisis de valores si existe columna valor
            if 'valor' in df.columns and not df['valor'].empty:
                report['valor_min'] = df['valor'].min()
                report['valor_max'] = df['valor'].max()
                report['valor_promedio'] = df['valor'].mean()
                report['valor_std'] = df['valor'].std()
            
            quality_report.append(report)
    
    return pd.DataFrame(quality_report)

# Generar reporte de calidad
quality_df = analyze_data_quality(all_data)

# Mostrar resumen por fuente
print("\n📊 ANÁLISIS DE CALIDAD POR FUENTE")
print("="*60)

for source in quality_df['fuente'].unique():
    source_data = quality_df[quality_df['fuente'] == source]
    
    print(f"\n🔍 {source}:")
    print(f"   Total series: {len(source_data)}")
    print(f"   Total registros: {source_data['registros'].sum():,}")
    print(f"   Promedio nulos: {source_data['pct_nulos'].mean():.2f}%")
    
    # Frecuencias
    freq_counts = source_data['frecuencia'].value_counts()
    if not freq_counts.empty:
        print(f"   Frecuencias:")
        for freq, count in freq_counts.items():
            print(f"      • {freq}: {count} series")

# Mostrar tabla detallada
print("\n📋 DETALLE DE SERIES TEMPORALES:")
print("="*60)

# Ordenar por fuente y serie
quality_df_sorted = quality_df.sort_values(['fuente', 'serie'])

# Seleccionar columnas clave para mostrar
display_cols = ['fuente', 'serie', 'registros', 'frecuencia', 'pct_nulos']
if 'fecha_inicio' in quality_df.columns:
    display_cols.extend(['fecha_inicio', 'fecha_fin'])

print(quality_df_sorted[display_cols].to_string(index=False))


## 🎯 Análisis Especial: World Bank Monthly Commodities

Análisis detallado de los datos MENSUALES de commodities del World Bank Pink Sheet, críticos para la predicción del precio del acero.


In [ ]:
# Análisis especial de World Bank Monthly Commodities
print("🎯 ANÁLISIS DE WORLD BANK MONTHLY COMMODITIES")
print("="*70)

# Verificar si tenemos datos de World Bank Monthly
wb_monthly_found = False
for source in all_data.keys():
    if 'world' in source.lower():
        # Buscar series mensuales de commodities
        monthly_series = {}
        for series_name, data in all_data[source].items():
            if isinstance(data, pd.DataFrame) and 'monthly' in series_name.lower():
                monthly_series[series_name] = data
                wb_monthly_found = True
        
        if monthly_series:
            print(f"\n✅ Encontradas {len(monthly_series)} series mensuales de commodities:")
            print("-"*50)
            
            # Analizar cada serie mensual
            commodities_summary = []
            for name, df in monthly_series.items():
                if 'fecha' in df.columns and 'valor' in df.columns:
                    # Limpiar nombre
                    commodity = name.replace('bank_monthly_', '').replace('_20250926', '')
                    
                    # Calcular estadísticas
                    stats = {
                        'Commodity': commodity.replace('_', ' ').title(),
                        'Puntos': len(df),
                        'Inicio': df['fecha'].min().strftime('%Y-%m'),
                        'Fin': df['fecha'].max().strftime('%Y-%m'),
                        'Último Valor': f"{df['valor'].iloc[-1]:.2f}",
                        'Promedio': f"{df['valor'].mean():.2f}",
                        'Volatilidad': f"{df['valor'].std():.2f}",
                        'Cambio %': f"{((df['valor'].iloc[-1] / df['valor'].iloc[0]) - 1) * 100:.1f}%"
                    }
                    commodities_summary.append(stats)
                    
                    # Mostrar info de cada commodity
                    print(f"\n📊 {stats['Commodity']}:")
                    print(f"   • Período: {stats['Inicio']} a {stats['Fin']} ({stats['Puntos']} meses)")
                    print(f"   • Último valor: ${stats['Último Valor']}")
                    print(f"   • Promedio histórico: ${stats['Promedio']}")
                    print(f"   • Volatilidad (std): ${stats['Volatilidad']}")
                    print(f"   • Cambio total: {stats['Cambio %']}")
            
            # Crear DataFrame resumen
            if commodities_summary:
                wb_monthly_df = pd.DataFrame(commodities_summary)
                print("\n📋 RESUMEN DE COMMODITIES MENSUALES:")
                print("-"*50)
                print(wb_monthly_df.to_string(index=False))
                
                # Identificar commodities críticos para acero
                critical_commodities = ['iron ore', 'coal', 'copper', 'aluminum', 'zinc']
                print("\n⭐ COMMODITIES CRÍTICOS PARA PRODUCCIÓN DE ACERO:")
                print("-"*50)
                for commodity in critical_commodities:
                    for name, df in monthly_series.items():
                        if commodity.replace(' ', '_') in name.lower():
                            if 'fecha' in df.columns and 'valor' in df.columns:
                                latest_val = df['valor'].iloc[-1]
                                prev_val = df['valor'].iloc[-12] if len(df) > 12 else df['valor'].iloc[0]
                                change_12m = ((latest_val / prev_val) - 1) * 100
                                
                                print(f"• {commodity.upper()}:")
                                print(f"  - Último precio: ${latest_val:.2f}")
                                print(f"  - Cambio 12 meses: {change_12m:+.1f}%")
                                
                                # Tendencia
                                if len(df) > 3:
                                    trend = "📈 Alcista" if df['valor'].iloc[-1] > df['valor'].iloc[-3] else "📉 Bajista"
                                    print(f"  - Tendencia 3 meses: {trend}")
            break

if not wb_monthly_found:
    print("\n⚠️ No se encontraron datos de World Bank Monthly Commodities")
    print("Verificando estructura de datos...")
    
    # Buscar en todas las fuentes
    for source, data in all_data.items():
        if 'world' in source.lower():
            print(f"\n📁 Fuente: {source}")
            series_list = [k for k in data.keys() if isinstance(data[k], pd.DataFrame)]
            print(f"   Series encontradas: {', '.join(series_list[:5])}...")


In [ ]:
# Visualización de commodities mensuales críticos
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear visualización de commodities críticos
fig_commodities = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Mineral de Hierro (Iron Ore)', 'Carbón Australiano (Coking Coal)',
        'Cobre (Copper)', 'Aluminio (Aluminum)',
        'Zinc', 'Níquel (Nickel)'
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.15
)

# Buscar y graficar series de World Bank Monthly
plot_positions = [
    ('iron_ore', 1, 1, 'red'),
    ('coal_australian', 1, 2, 'black'),
    ('copper', 2, 1, 'orange'),
    ('aluminum', 2, 2, 'silver'),
    ('zinc', 3, 1, 'blue'),
    ('nickel', 3, 2, 'green')
]

series_found = False
for source, data in all_data.items():
    if 'world' in source.lower():
        for series_name, df in data.items():
            if isinstance(df, pd.DataFrame) and 'monthly' in series_name.lower():
                # Buscar cada commodity
                for commodity, row, col, color in plot_positions:
                    if commodity in series_name.lower():
                        if 'fecha' in df.columns and 'valor' in df.columns:
                            fig_commodities.add_trace(
                                go.Scatter(
                                    x=df['fecha'],
                                    y=df['valor'],
                                    mode='lines+markers',
                                    name=commodity.replace('_', ' ').title(),
                                    line=dict(color=color, width=2),
                                    marker=dict(size=4),
                                    showlegend=False
                                ),
                                row=row, col=col
                            )
                            
                            # Agregar línea de tendencia
                            if len(df) > 12:
                                # Media móvil de 3 meses
                                df['ma3'] = df['valor'].rolling(window=3, center=True).mean()
                                fig_commodities.add_trace(
                                    go.Scatter(
                                        x=df['fecha'],
                                        y=df['ma3'],
                                        mode='lines',
                                        name='MA3',
                                        line=dict(color=color, width=1, dash='dash'),
                                        opacity=0.5,
                                        showlegend=False
                                    ),
                                    row=row, col=col
                                )
                            series_found = True

if series_found:
    # Actualizar layout
    fig_commodities.update_layout(
        height=900,
        title_text="📊 World Bank Pink Sheet - Commodities Mensuales Críticos para Acero",
        title_font_size=16,
        showlegend=False,
        hovermode='x unified'
    )
    
    # Actualizar ejes
    fig_commodities.update_xaxes(title_text="Fecha", tickformat="%Y-%m")
    fig_commodities.update_yaxes(title_text="USD")
    
    # Mostrar figura
    fig_commodities.show()
    print("✅ Visualización de commodities mensuales generada")
else:
    print("⚠️ No se encontraron datos mensuales de commodities para visualizar")


## 📊 Análisis Completo de Series Mensuales por Fuente

Análisis detallado de TODAS las series con frecuencia mensual, agrupadas por fuente de datos.


In [ ]:
# Visualización de Series Temporales Clave - SOLO DATOS POST-2025
# IDENTIFICACIÓN DE VARIABLE OBJETIVO

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

print("🎯 IDENTIFICACIÓN DE VARIABLE OBJETIVO")
print("="*80)
print("\n📌 VARIABLE OBJETIVO PRINCIPAL:")
print("   • LME steel_rebar: Precio internacional de varilla corrugada (steel rebar)")
print("   • Fuente: London Metal Exchange (LME)")
print("\n📌 VARIABLES OBJETIVO SECUNDARIAS/PROXY:")
print("   • AHMSA ahmsa: Precio de acero AHMSA México")
print("   • LME steel_etf: ETF de acero como proxy")
print("\n" + "="*80)

# Fecha de corte
fecha_corte = pd.Timestamp('2025-01-01')

# Función para normalizar fechas
def normalizar_fecha(fecha):
    if pd.notna(fecha):
        if hasattr(fecha, 'tz') and fecha.tz is not None:
            return fecha.tz_localize(None)
    return fecha

# Función para filtrar datos post-2025
def filtrar_post_2025(df):
    # Verificar si es un DataFrame
    if not isinstance(df, pd.DataFrame):
        return None
    if df.empty:
        return None
    df_copy = df.copy()
    df_copy['fecha'] = df_copy['fecha'].apply(normalizar_fecha)
    return df_copy[pd.to_datetime(df_copy['fecha']) > fecha_corte]

# Función para obtener valor de cierre
def obtener_valor_cierre(df, fuente):
    if not isinstance(df, pd.DataFrame):
        return None
    
    # Prioridad de columnas según la fuente
    if 'Close' in df.columns:
        return df['Close']
    elif 'precio_cierre' in df.columns:
        return df['precio_cierre']
    elif 'valor' in df.columns:
        return df['valor']
    elif 'close' in df.columns:
        return df['close']
    
    # Si no encuentra ninguna, buscar cualquier columna numérica que no sea fecha
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    if len(numeric_cols) > 0:
        # Excluir columnas que parecen ser índices o fechas
        valid_cols = [col for col in numeric_cols if 'fecha' not in col.lower() and 'date' not in col.lower() and 'index' not in col.lower()]
        if valid_cols:
            return df[valid_cols[0]]
    
    return None

# Crear figura con subplots
fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=(
        '🎯 VARIABLE OBJETIVO: Precio Varilla LME', 'Precio AHMSA México (Proxy Local)',
        'Tipo de Cambio USD/MXN', 'Precios Metales Base (LME)',
        'Mineral de Hierro (Raw Materials)', 'Indicadores FRED (US)',
        'Tasas de Interés (Banxico)', 'Indicadores Construcción (INEGI)'
    ),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]],
    vertical_spacing=0.08,
    horizontal_spacing=0.12
)

series_agregadas = []
estadisticas = {}

# 1. VARIABLE OBJETIVO PRINCIPAL: Precio de varilla LME
print("\n📊 ANÁLISIS DE VARIABLE OBJETIVO:")
print("-"*60)

if 'LME' in all_data:
    # Buscar steel_rebar
    steel_rebar_keys = [k for k in all_data['LME'].keys() if 'steel_rebar' in k.lower()]
    
    for key in steel_rebar_keys:
        df = all_data['LME'][key]
        # Verificar que sea un DataFrame antes de procesar
        if not isinstance(df, pd.DataFrame):
            continue
        df_2025 = filtrar_post_2025(df)
        if df_2025 is not None and not df_2025.empty:
            valores = obtener_valor_cierre(df_2025, 'LME')
            if valores is not None:
                fig.add_trace(
                    go.Scatter(
                        x=df_2025['fecha'], 
                        y=valores, 
                        name='🎯 Varilla LME (OBJETIVO)', 
                        line=dict(color='red', width=3),
                        mode='lines+markers',
                        marker=dict(size=3)
                    ),
                    row=1, col=1
                )
                estadisticas['LME_steel_rebar'] = {
                    'puntos': len(df_2025),
                    'ultimo_valor': valores.iloc[-1],
                    'fecha_max': df_2025['fecha'].max(),
                    'promedio': valores.mean(),
                    'volatilidad': valores.std()
                }
                print(f"✅ VARIABLE OBJETIVO ENCONTRADA: LME {key}")
                print(f"   • Puntos de datos en 2025: {len(df_2025)}")
                print(f"   • Último valor: ${valores.iloc[-1]:.2f}")
                print(f"   • Promedio 2025: ${valores.mean():.2f}")
                print(f"   • Volatilidad: {valores.std():.2f}")

# 2. AHMSA - Precio local México
if 'ahmsa' in all_data:
    ahmsa_keys = [k for k in all_data['ahmsa'].keys() if 'ahmsa' in k.lower()]
    
    for key in ahmsa_keys[:1]:  # Solo el primero
        df = all_data['ahmsa'][key]
        if not isinstance(df, pd.DataFrame):
            continue
        df_2025 = filtrar_post_2025(df)
        if df_2025 is not None and not df_2025.empty:
            valores = obtener_valor_cierre(df_2025, 'ahmsa')
            if valores is not None:
                fig.add_trace(
                    go.Scatter(
                        x=df_2025['fecha'], 
                        y=valores, 
                        name='AHMSA México', 
                        line=dict(color='orange', width=2)
                    ),
                    row=1, col=2
                )
                estadisticas['AHMSA'] = {
                    'puntos': len(df_2025),
                    'ultimo_valor': valores.iloc[-1]
                }
                series_agregadas.append(f"AHMSA: {len(df_2025)} puntos")

# 3. Tipo de cambio USD/MXN
if 'banxico' in all_data:
    for key in ['usd_mxn_20250926', 'SF60653']:
        if key in all_data['banxico']:
            df = all_data['banxico'][key]
            if not isinstance(df, pd.DataFrame):
                continue
            df_2025 = filtrar_post_2025(df)
            if df_2025 is not None and not df_2025.empty:
                valores = obtener_valor_cierre(df_2025, 'banxico')
                if valores is not None:
                    fig.add_trace(
                        go.Scatter(
                            x=df_2025['fecha'], 
                            y=valores, 
                            name='USD/MXN', 
                            line=dict(color='green', width=2)
                        ),
                        row=2, col=1
                    )
                    series_agregadas.append(f"USD/MXN: {len(df_2025)} puntos")
                    break

# 4. Precios de metales base (LME)
if 'LME' in all_data:
    metales = ['Aluminio', 'Cobre', 'Zinc', 'iron_ore']
    colors = ['blue', 'brown', 'purple', 'gray']
    
    for i, metal in enumerate(metales):
        for key in all_data['LME'].keys():
            if metal.lower() in key.lower():
                df = all_data['LME'][key]
                if not isinstance(df, pd.DataFrame):
                    continue
                df_2025 = filtrar_post_2025(df)
                if df_2025 is not None and not df_2025.empty:
                    valores = obtener_valor_cierre(df_2025, 'LME')
                    if valores is not None:
                        fig.add_trace(
                            go.Scatter(
                                x=df_2025['fecha'], 
                                y=valores, 
                                name=metal, 
                                line=dict(color=colors[i % len(colors)], width=1.5)
                            ),
                            row=2, col=2
                        )
                        series_agregadas.append(f"{metal}: {len(df_2025)} puntos")
                        break

# 5. Mineral de hierro - Raw Materials
if 'RawMaterials' in all_data:
    iron_keys = [k for k in all_data['RawMaterials'].keys() if 'MineralHierro' in k or 'iron' in k.lower()]
    
    for i, key in enumerate(iron_keys[:3]):  # Máximo 3 series
        df = all_data['RawMaterials'][key]
        if not isinstance(df, pd.DataFrame):
            continue
        df_2025 = filtrar_post_2025(df)
        if df_2025 is not None and not df_2025.empty:
            valores = obtener_valor_cierre(df_2025, 'RawMaterials')
            if valores is not None:
                fig.add_trace(
                    go.Scatter(
                        x=df_2025['fecha'], 
                        y=valores, 
                        name=key.split('_')[1][:10], 
                        line=dict(width=1.5)
                    ),
                    row=3, col=1
                )
                series_agregadas.append(f"{key}: {len(df_2025)} puntos")

# 6. Indicadores FRED
if 'FRED' in all_data:
    fred_keys = ['steel_production', 'iron_steel_scrap', 'ppi_metals', 'federal_funds_rate']
    
    for key_partial in fred_keys:
        for key in all_data['FRED'].keys():
            if key_partial in key.lower():
                df = all_data['FRED'][key]
                if not isinstance(df, pd.DataFrame):
                    continue
                df_2025 = filtrar_post_2025(df)
                if df_2025 is not None and not df_2025.empty:
                    valores = obtener_valor_cierre(df_2025, 'FRED')
                    if valores is not None:
                        fig.add_trace(
                            go.Scatter(
                                x=df_2025['fecha'], 
                                y=valores, 
                                name=key_partial.replace('_', ' ').title()[:15], 
                                line=dict(width=1.5)
                            ),
                            row=3, col=2
                        )
                        series_agregadas.append(f"FRED {key_partial}: {len(df_2025)} puntos")
                        break

# 7. Tasas de interés - Banxico
if 'banxico' in all_data:
    tasas = ['tiie_28', 'interest_rate', 'TIIE28']
    
    for tasa in tasas:
        for key in all_data['banxico'].keys():
            if tasa.lower() in key.lower():
                df = all_data['banxico'][key]
                if not isinstance(df, pd.DataFrame):
                    continue
                df_2025 = filtrar_post_2025(df)
                if df_2025 is not None and not df_2025.empty:
                    valores = obtener_valor_cierre(df_2025, 'banxico')
                    if valores is not None:
                        fig.add_trace(
                            go.Scatter(
                                x=df_2025['fecha'], 
                                y=valores, 
                                name=tasa.upper(), 
                                line=dict(width=1.5)
                            ),
                            row=4, col=1
                        )
                        series_agregadas.append(f"{tasa}: {len(df_2025)} puntos")
                        break

# 8. Indicadores construcción - INEGI
if 'INEGI' in all_data:
    construccion_keys = ['ProduccionConstruccion', 'inpp_construccion']
    
    for key_partial in construccion_keys:
        for key in all_data['INEGI'].keys():
            if key_partial in key:
                df = all_data['INEGI'][key]
                if not isinstance(df, pd.DataFrame):
                    continue
                df_2025 = filtrar_post_2025(df)
                if df_2025 is not None and not df_2025.empty:
                    valores = obtener_valor_cierre(df_2025, 'INEGI')
                    if valores is not None:
                        fig.add_trace(
                            go.Scatter(
                                x=df_2025['fecha'], 
                                y=valores, 
                                name=key_partial.replace('_', ' ').title()[:20], 
                                line=dict(width=1.5)
                            ),
                            row=4, col=2
                        )
                        series_agregadas.append(f"INEGI {key_partial}: {len(df_2025)} puntos")
                        break

# Actualizar layout
fig.update_layout(
    height=1400,
    showlegend=True,
    title_text="📊 Series Temporales Clave (Solo Datos Post-2025) - Variable Objetivo: LME Steel Rebar",
    title_font_size=16,
    hovermode='x unified',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(size=9)
    )
)

# Actualizar todos los ejes X
for i in range(1, 5):
    for j in range(1, 3):
        fig.update_xaxes(tickformat="%Y-%m", tickangle=45, row=i, col=j)
        fig.update_yaxes(title_text="Valor", row=i, col=j)

# Mostrar figura
fig.show()

# Resumen de series agregadas
print("\n📊 RESUMEN DE SERIES VISUALIZADAS:")
print("-"*60)
for serie in series_agregadas:
    print(f"   • {serie}")

print(f"\n✅ Total de series con datos post-2025: {len(series_agregadas)}")

# Análisis de correlación potencial
if 'LME_steel_rebar' in estadisticas:
    print("\n🔍 ANÁLISIS DE VARIABLE OBJETIVO:")
    print("-"*60)
    print("Variable: LME Steel Rebar (Precio Internacional de Varilla Corrugada)")
    stats = estadisticas['LME_steel_rebar']
    print(f"   • Último precio: ${stats['ultimo_valor']:.2f}")
    print(f"   • Promedio 2025: ${stats['promedio']:.2f}")
    print(f"   • Volatilidad (std): ${stats['volatilidad']:.2f}")
    print(f"   • Coeficiente de variación: {(stats['volatilidad']/stats['promedio']*100):.2f}%")
    print(f"   • Última actualización: {stats['fecha_max'].strftime('%Y-%m-%d')}")
    
    print("\n💡 RECOMENDACIONES PARA MODELADO:")
    print("-"*60)
    print("1. Variable objetivo principal: LME steel_rebar (precio internacional)")
    print("2. Variables explicativas clave:")
    print("   • Tipo de cambio USD/MXN (impacto directo en precio local)")
    print("   • Precios de metales base (correlación con mercado de commodities)")
    print("   • Mineral de hierro (materia prima principal)")
    print("   • Indicadores de construcción (demanda)")
    print("   • Tasas de interés (costo financiero)")
    print("3. Considerar rezagos de 1-3 días para capturar efectos retardados")
    print("4. Normalizar todas las series antes del modelado")

print("\n" + "="*80)


In [ ]:
# Análisis exhaustivo de todas las series mensuales
print("📊 ANÁLISIS COMPLETO DE SERIES MENSUALES POR FUENTE")
print("="*80)

# Primero, identificar todas las series mensuales
monthly_series_by_source = {}

# Usar quality_df que ya tiene la información de frecuencia
if 'quality_df' in locals() and not quality_df.empty:
    # Filtrar solo series mensuales
    monthly_df = quality_df[quality_df['frecuencia'] == 'Mensual'].copy()
    
    if not monthly_df.empty:
        print(f"\n📈 Total de series mensuales encontradas: {len(monthly_df)}")
        print(f"📁 Fuentes con datos mensuales: {monthly_df['fuente'].nunique()}")
        
        # Agrupar por fuente
        for source in monthly_df['fuente'].unique():
            source_monthly = monthly_df[monthly_df['fuente'] == source]
            monthly_series_by_source[source] = source_monthly
            
        # Resumen general
        print("\n📋 RESUMEN POR FUENTE:")
        print("-"*60)
        for source, series_df in monthly_series_by_source.items():
            print(f"• {source}: {len(series_df)} series mensuales")
    else:
        print("⚠️ No se encontraron series con frecuencia mensual")
else:
    print("⚠️ No se ha ejecutado el análisis de calidad previo")

print("\n" + "="*80)


In [ ]:
# Análisis detallado por cada fuente con series mensuales
if monthly_series_by_source:
    for source_name, source_series in monthly_series_by_source.items():
        print(f"\n{'='*80}")
        print(f"📊 FUENTE: {source_name}")
        print(f"{'='*80}")
        
        # Información general de la fuente
        print(f"\n📈 Estadísticas Generales:")
        print(f"   • Total series mensuales: {len(source_series)}")
        print(f"   • Total puntos de datos: {source_series['registros'].sum():,}")
        
        if 'fecha_inicio' in source_series.columns and 'fecha_fin' in source_series.columns:
            # Encontrar el rango temporal general
            fecha_min = source_series['fecha_inicio'].min()
            fecha_max = source_series['fecha_fin'].max()
            if pd.notna(fecha_min) and pd.notna(fecha_max):
                print(f"   • Período cubierto: {fecha_min.strftime('%Y-%m-%d')} a {fecha_max.strftime('%Y-%m-%d')}")
                print(f"   • Duración total: {(fecha_max - fecha_min).days} días (~{(fecha_max - fecha_min).days // 30} meses)")
        
        # Análisis por cada serie
        print(f"\n📋 Detalle de Series:")
        print("-"*60)
        
        for idx, row in source_series.iterrows():
            serie_name = row['serie']
            
            # Obtener el DataFrame real de la serie
            if source_name in all_data and serie_name in all_data[source_name]:
                df = all_data[source_name][serie_name]
                
                if isinstance(df, pd.DataFrame):
                    print(f"\n📌 {serie_name}:")
                    print(f"   • Registros: {len(df)}")
                    
                    # Análisis temporal
                    if 'fecha' in df.columns and not df.empty:
                        fecha_min = df['fecha'].min()
                        fecha_max = df['fecha'].max()
                        print(f"   • Período: {fecha_min.strftime('%Y-%m')} a {fecha_max.strftime('%Y-%m')}")
                        
                        # Calcular gaps o datos faltantes
                        expected_months = pd.date_range(start=fecha_min, end=fecha_max, freq='MS')
                        actual_months = pd.to_datetime(df['fecha']).dt.to_period('M').unique()
                        missing_months = len(expected_months) - len(actual_months)
                        
                        if missing_months > 0:
                            print(f"   • ⚠️ Meses faltantes: {missing_months}")
                        else:
                            print(f"   • ✅ Serie completa sin gaps")
                    
                    # Análisis de valores
                    if 'valor' in df.columns and not df['valor'].empty:
                        valores = df['valor'].dropna()
                        if len(valores) > 0:
                            print(f"   • Valor mínimo: {valores.min():.2f}")
                            print(f"   • Valor máximo: {valores.max():.2f}")
                            print(f"   • Valor promedio: {valores.mean():.2f}")
                            print(f"   • Desviación estándar: {valores.std():.2f}")
                            print(f"   • Coef. variación: {(valores.std() / valores.mean() * 100):.1f}%")
                            
                            # Tendencia reciente (últimos 6 meses)
                            if len(valores) >= 6:
                                recent_values = valores.tail(6)
                                older_values = valores.tail(12).head(6) if len(valores) >= 12 else valores.head(6)
                                
                                change_pct = ((recent_values.mean() - older_values.mean()) / older_values.mean() * 100)
                                
                                if change_pct > 5:
                                    trend = "📈 Tendencia alcista"
                                elif change_pct < -5:
                                    trend = "📉 Tendencia bajista"
                                else:
                                    trend = "➡️ Tendencia lateral"
                                
                                print(f"   • Tendencia 6 meses: {trend} ({change_pct:+.1f}%)")
                            
                            # Último valor disponible
                            print(f"   • Último valor: {valores.iloc[-1]:.2f} ({df['fecha'].iloc[-1].strftime('%Y-%m') if 'fecha' in df.columns else 'N/A'})")
                    
                    # Metadata si existe
                    metadata_key = f"{serie_name}_metadata"
                    if metadata_key in all_data[source_name]:
                        metadata = all_data[source_name][metadata_key]
                        if isinstance(metadata, dict):
                            if 'unit' in metadata:
                                print(f"   • Unidad: {metadata['unit']}")
                            if 'source' in metadata:
                                print(f"   • Origen: {metadata['source']}")
            else:
                print(f"\n📌 {serie_name}: [Datos no disponibles]")
        
        print()  # Línea en blanco entre fuentes


### 📋 Resumen Ejecutivo: Series Mensuales, Semanales y Diarias Críticas para Predicción


In [ ]:
# VERSIÓN CORREGIDA - Análisis de series por frecuencia con datos posteriores a 2025
# Corrige el problema de comparación entre fechas con y sin timezone

print("📊 ANÁLISIS DE SERIES CON DATOS POSTERIORES A ENERO 2025")
print("="*80)

# Fecha de corte (sin timezone para compatibilidad)
fecha_corte = pd.Timestamp('2025-01-01')

# Función helper para normalizar fechas
def normalizar_fecha(fecha):
    """Elimina timezone de una fecha si lo tiene"""
    if pd.notna(fecha):
        if hasattr(fecha, 'tz') and fecha.tz is not None:
            return fecha.tz_localize(None)
    return fecha

# ============================================
# SERIES MENSUALES
# ============================================
print("\n📅 SERIES MENSUALES")
print("-"*60)

series_mensuales_2025 = []

if 'quality_df' in locals():
    # Filtrar solo series mensuales
    monthly_df = quality_df[quality_df['frecuencia'] == 'Mensual'].copy()
    
    for idx, row in monthly_df.iterrows():
        # Normalizar fecha_fin para comparación
        fecha_fin = normalizar_fecha(row['fecha_fin'])
        
        if pd.notna(fecha_fin) and fecha_fin > fecha_corte:
            source_name = row['fuente']
            serie_name = row['serie']
            
            if source_name in all_data and serie_name in all_data[source_name]:
                df = all_data[source_name][serie_name]
                if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
                    # Normalizar fechas en el DataFrame
                    df_copy = df.copy()
                    df_copy['fecha'] = df_copy['fecha'].apply(normalizar_fecha)
                    
                    # Contar registros posteriores a enero 2025
                    df_2025 = df_copy[pd.to_datetime(df_copy['fecha']) > fecha_corte]
                    
                    if not df_2025.empty:
                        series_mensuales_2025.append({
                            'fuente': source_name,
                            'serie': serie_name,
                            'fecha_inicio': normalizar_fecha(row['fecha_inicio']),
                            'fecha_fin': fecha_fin,
                            'registros_totales': row['registros'],
                            'registros_2025': len(df_2025),
                            'fecha_max_2025': df_2025['fecha'].max(),
                            'ultimo_valor': df_2025['valor'].iloc[-1] if 'valor' in df_2025.columns else None
                        })

print(f"✅ Series mensuales con datos post-2025: {len(series_mensuales_2025)}")
if series_mensuales_2025:
    for serie in series_mensuales_2025[:5]:  # Mostrar primeras 5
        print(f"   • [{serie['fuente']}] {serie['serie']}: hasta {serie['fecha_max_2025'].strftime('%Y-%m-%d')}")

# ============================================
# SERIES SEMANALES
# ============================================
print("\n📅 SERIES SEMANALES")
print("-"*60)

series_semanales_2025 = []

if 'quality_df' in locals():
    # Filtrar solo series semanales
    weekly_df = quality_df[quality_df['frecuencia'] == 'Semanal'].copy()
    
    for idx, row in weekly_df.iterrows():
        # Normalizar fecha_fin para comparación
        fecha_fin = normalizar_fecha(row['fecha_fin'])
        
        if pd.notna(fecha_fin) and fecha_fin > fecha_corte:
            source_name = row['fuente']
            serie_name = row['serie']
            
            if source_name in all_data and serie_name in all_data[source_name]:
                df = all_data[source_name][serie_name]
                if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
                    # Normalizar fechas en el DataFrame
                    df_copy = df.copy()
                    df_copy['fecha'] = df_copy['fecha'].apply(normalizar_fecha)
                    
                    # Contar registros posteriores a enero 2025
                    df_2025 = df_copy[pd.to_datetime(df_copy['fecha']) > fecha_corte]
                    
                    if not df_2025.empty:
                        series_semanales_2025.append({
                            'fuente': source_name,
                            'serie': serie_name,
                            'fecha_inicio': normalizar_fecha(row['fecha_inicio']),
                            'fecha_fin': fecha_fin,
                            'registros_totales': row['registros'],
                            'registros_2025': len(df_2025),
                            'fecha_max_2025': df_2025['fecha'].max(),
                            'ultimo_valor': df_2025['valor'].iloc[-1] if 'valor' in df_2025.columns else None
                        })

print(f"✅ Series semanales con datos post-2025: {len(series_semanales_2025)}")
if series_semanales_2025:
    for serie in series_semanales_2025[:5]:  # Mostrar primeras 5
        print(f"   • [{serie['fuente']}] {serie['serie']}: hasta {serie['fecha_max_2025'].strftime('%Y-%m-%d')}")
else:
    print("   ⚠️ No hay series semanales con datos recientes")

# ============================================
# SERIES DIARIAS
# ============================================
print("\n📅 SERIES DIARIAS")
print("-"*60)

series_diarias_2025 = []

if 'quality_df' in locals():
    # Filtrar solo series diarias
    daily_df = quality_df[quality_df['frecuencia'] == 'Diaria'].copy()
    
    for idx, row in daily_df.iterrows():
        # Normalizar fecha_fin para comparación
        fecha_fin = normalizar_fecha(row['fecha_fin'])
        
        if pd.notna(fecha_fin) and fecha_fin > fecha_corte:
            source_name = row['fuente']
            serie_name = row['serie']
            
            if source_name in all_data and serie_name in all_data[source_name]:
                df = all_data[source_name][serie_name]
                if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
                    # Normalizar fechas en el DataFrame
                    df_copy = df.copy()
                    df_copy['fecha'] = df_copy['fecha'].apply(normalizar_fecha)
                    
                    # Contar registros posteriores a enero 2025
                    df_2025 = df_copy[pd.to_datetime(df_copy['fecha']) > fecha_corte]
                    
                    if not df_2025.empty:
                        series_diarias_2025.append({
                            'fuente': source_name,
                            'serie': serie_name,
                            'fecha_inicio': normalizar_fecha(row['fecha_inicio']),
                            'fecha_fin': fecha_fin,
                            'registros_totales': row['registros'],
                            'registros_2025': len(df_2025),
                            'fecha_max_2025': df_2025['fecha'].max(),
                            'ultimo_valor': df_2025['valor'].iloc[-1] if 'valor' in df_2025.columns else None
                        })

# Ordenar por fecha más reciente
series_diarias_2025 = sorted(series_diarias_2025, key=lambda x: x['fecha_max_2025'], reverse=True)

print(f"✅ Series diarias con datos post-2025: {len(series_diarias_2025)}")

if series_diarias_2025:
    # Agrupar por fuente
    fuentes_agrupadas = {}
    for serie in series_diarias_2025:
        if serie['fuente'] not in fuentes_agrupadas:
            fuentes_agrupadas[serie['fuente']] = []
        fuentes_agrupadas[serie['fuente']].append(serie)
    
    # Mostrar resumen por fuente
    print("\n📊 Resumen por fuente de datos:")
    for fuente, series in fuentes_agrupadas.items():
        total_registros = sum(s['registros_2025'] for s in series)
        print(f"   • {fuente}: {len(series)} series, {total_registros:,} registros en 2025")
    
    # TOP 5 series con más datos
    print("\n🏆 TOP 5 series diarias con más datos en 2025:")
    top_series = sorted(series_diarias_2025, key=lambda x: x['registros_2025'], reverse=True)[:5]
    for i, serie in enumerate(top_series, 1):
        print(f"   {i}. [{serie['fuente']}] {serie['serie']}: {serie['registros_2025']} registros")

# ============================================
# RESUMEN FINAL
# ============================================
print("\n" + "="*80)
print("🎯 RESUMEN FINAL - SERIES CON DATOS POST-2025")
print("="*80)

total_mensuales = len(series_mensuales_2025)
total_semanales = len(series_semanales_2025)
total_diarias = len(series_diarias_2025)
total_general = total_mensuales + total_semanales + total_diarias

print(f"\n📊 Distribución por frecuencia:")
print(f"   • Mensuales: {total_mensuales} series")
print(f"   • Semanales: {total_semanales} series")
print(f"   • Diarias: {total_diarias} series")
print(f"\n✅ TOTAL: {total_general} series con datos actualizados más allá de enero 2025")

if total_diarias > 0:
    total_registros_2025 = sum(s['registros_2025'] for s in series_diarias_2025)
    print(f"\n📈 Total de registros diarios en 2025: {total_registros_2025:,}")
    
print("\n" + "="*80)


## 📈 Visualización de Series Temporales Clave

Visualizamos las series más importantes para la predicción del precio de varilla corrugada.


In [ ]:
# Crear visualización comparativa de todas las series mensuales con datos recientes
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

if monthly_series_by_source:
    # Fecha de corte: enero 2025
    fecha_corte = pd.Timestamp('2025-01-01')
    
    # Filtrar series que tienen datos más allá de enero 2025
    series_recientes = []
    
    print("🔍 Filtrando series con datos más allá de enero 2025...")
    
    for source_name in sorted(monthly_series_by_source.keys()):
        source_series = monthly_series_by_source[source_name]
        for idx, row in source_series.iterrows():
            serie_name = row['serie']
            if source_name in all_data and serie_name in all_data[source_name]:
                df = all_data[source_name][serie_name]
                if isinstance(df, pd.DataFrame) and 'fecha' in df.columns and 'valor' in df.columns:
                    # Verificar si tiene datos después de enero 2025
                    if not df.empty and pd.to_datetime(df['fecha'].max()) > fecha_corte:
                        series_recientes.append({
                            'source': source_name,
                            'serie': serie_name,
                            'df': df,
                            'fecha_max': df['fecha'].max(),
                            'registros_2025': len(df[pd.to_datetime(df['fecha']) > fecha_corte])
                        })
    
    # Informar cuántas series cumplen el criterio
    print(f"✅ Series con datos después de enero 2025: {len(series_recientes)}")
    print(f"❌ Series excluidas (sin datos recientes): {sum(len(s) for s in monthly_series_by_source.values()) - len(series_recientes)}")
    
    if series_recientes:
        # Ordenar por fecha máxima (más recientes primero)
        series_recientes = sorted(series_recientes, key=lambda x: x['fecha_max'], reverse=True)
        
        # Mostrar resumen de series incluidas
        print("\n📊 Series incluidas en la visualización:")
        print("-" * 60)
        for s in series_recientes[:10]:  # Mostrar las 10 más recientes
            fecha_str = s['fecha_max'].strftime('%Y-%m-%d') if hasattr(s['fecha_max'], 'strftime') else str(s['fecha_max'])
            print(f"  • [{s['source']}] {s['serie'][:30]}: hasta {fecha_str} ({s['registros_2025']} registros en 2025)")
        if len(series_recientes) > 10:
            print(f"  ... y {len(series_recientes) - 10} series más")
        
        # Determinar número de filas y columnas para subplots
        n_cols = 3
        n_rows = (len(series_recientes) + n_cols - 1) // n_cols
        
        # Preparar lista de títulos
        subplot_titles_list = []
        for s in series_recientes:
            fecha_str = s['fecha_max'].strftime('%m/%Y') if hasattr(s['fecha_max'], 'strftime') else str(s['fecha_max'])[:7]
            subplot_titles_list.append(f"{s['source']}: {s['serie'][:20]} (hasta {fecha_str})")
    
        # Rellenar con títulos vacíos si es necesario para completar la grid
        while len(subplot_titles_list) < n_rows * n_cols:
            subplot_titles_list.append("")
        
        # Crear figura con subplots y títulos
        fig_monthly = make_subplots(
            rows=n_rows, 
            cols=n_cols,
            subplot_titles=subplot_titles_list[:n_rows * n_cols],  # Usar solo los títulos necesarios
            vertical_spacing=0.12,  # Aumentar espacio para títulos
            horizontal_spacing=0.1
        )
        
        # Colores por fuente
        color_map = {
            'FRED': 'blue',
            'INEGI': 'green',
            'banxico': 'red',
            'world': 'purple',
            'ahmsa': 'orange',
            'trading': 'brown',
            'YahooFinance': 'pink',
            'LME': 'gray',
            'RawMaterials': 'cyan'
        }
        
        plot_idx = 0
        
        # Iterar por las series filtradas con datos recientes
        for serie_info in series_recientes:
            source_name = serie_info['source']
            serie_name = serie_info['serie']
            df = serie_info['df']
            color = color_map.get(source_name, 'black')
            
            plot_idx += 1
            row_idx = (plot_idx - 1) // n_cols + 1
            col_idx = (plot_idx - 1) % n_cols + 1
            
            # Filtrar solo datos desde 2020 para mejor visualización
            df_filtered = df[pd.to_datetime(df['fecha']) >= '2020-01-01'].copy()
            
            if not df_filtered.empty:
                # Normalizar valores para comparación (0-100)
                valores = df_filtered['valor'].dropna()
                if len(valores) > 0 and valores.max() != valores.min():
                    valores_norm = (valores - valores.min()) / (valores.max() - valores.min()) * 100
                    
                    # Crear DataFrame temporal con índice alineado
                    df_temp = df_filtered[['fecha']].copy()
                    df_temp['valor_norm'] = valores_norm.values
                    
                    # Resaltar datos de 2025
                    df_2025 = df_temp[pd.to_datetime(df_temp['fecha']) > fecha_corte]
                    df_pre_2025 = df_temp[pd.to_datetime(df_temp['fecha']) <= fecha_corte]
                    
                    # Agregar traza para datos anteriores a 2025
                    if not df_pre_2025.empty:
                        fig_monthly.add_trace(
                            go.Scatter(
                                x=df_pre_2025['fecha'],
                                y=df_pre_2025['valor_norm'],
                                mode='lines',
                                name=f"{source_name}: {serie_name[:20]}",
                                line=dict(color=color, width=1.5),
                                showlegend=False,
                                hovertemplate=f"<b>{serie_name}</b><br>Fecha: %{{x|%Y-%m}}<br>Valor Norm: %{{y:.1f}}<extra></extra>"
                            ),
                            row=row_idx, col=col_idx
                        )
                    
                    # Agregar traza para datos de 2025 (resaltados)
                    if not df_2025.empty:
                        fig_monthly.add_trace(
                            go.Scatter(
                                x=df_2025['fecha'],
                                y=df_2025['valor_norm'],
                                mode='lines+markers',
                                name=f"{source_name}: {serie_name[:20]} (2025)",
                                line=dict(color=color, width=2.5, dash='solid'),
                                marker=dict(size=6, color=color),
                                showlegend=False,
                                hovertemplate=f"<b>{serie_name} (2025)</b><br>Fecha: %{{x|%Y-%m}}<br>Valor Norm: %{{y:.1f}}<extra></extra>"
                            ),
                            row=row_idx, col=col_idx
                        )
                    
                    # Actualizar ejes
                    fig_monthly.update_xaxes(
                        tickformat="%Y-%m",
                        tickangle=45,
                        row=row_idx, col=col_idx
                    )
                    fig_monthly.update_yaxes(
                        title_text="Norm",
                        row=row_idx, col=col_idx
                    )
    
        # Actualizar layout general
        fig_monthly.update_layout(
            height=250 * n_rows,  # Altura ajustada
            title_text="📊 Series Mensuales con Datos Posteriores a Enero 2025 (Valores Normalizados 0-100)",
            title_font_size=16,
            showlegend=False,
            hovermode='x'
        )
        
        # Agregar anotación explicativa
        fig_monthly.add_annotation(
            text="Nota: Los datos de 2025 se muestran con línea más gruesa y marcadores",
            xref="paper",
            yref="paper",
            x=0.5,
            y=-0.02,
            showarrow=False,
            font=dict(size=10, color="gray"),
            xanchor='center'
        )
        
        # Mostrar figura
        fig_monthly.show()
        
        print(f"\n✅ Visualización generada con {plot_idx} series mensuales con datos recientes")
        print("📌 Los datos posteriores a enero 2025 están resaltados con línea más gruesa y marcadores")
    else:
        print("\n⚠️ No se encontraron series mensuales con datos posteriores a enero 2025")
        print("   Esto podría indicar que los datos están desactualizados o que la fecha actual del sistema es anterior a 2025")
else:
    print("⚠️ No hay series mensuales para visualizar")


In [ ]:
# Visualización de series temporales DIARIAS con datos posteriores a 2025
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import pandas as pd

print("📊 VISUALIZACIÓN DE SERIES DIARIAS CON DATOS POST-2025")
print("="*80)

# Fecha de corte
fecha_corte = pd.Timestamp('2025-01-01')

# Función para normalizar fechas (eliminar timezone)
def normalizar_fecha(fecha):
    """Elimina timezone de una fecha si lo tiene"""
    if pd.notna(fecha):
        if hasattr(fecha, 'tz') and fecha.tz is not None:
            return fecha.tz_localize(None)
    return fecha

# Función para obtener el valor de cierre según la fuente
def obtener_valor_cierre(df, fuente):
    """Obtiene la columna de valor de cierre según la fuente de datos"""
    if fuente in ['ahmsa', 'LME', 'YahooFinance']:
        if 'Close' in df.columns:
            return df['Close']
    elif fuente == 'RawMaterials':
        if 'precio_cierre' in df.columns:
            return df['precio_cierre']
    
    # Fallback a otras columnas comunes
    if 'valor' in df.columns:
        return df['valor']
    elif 'close' in df.columns:
        return df['close']
    elif 'Close' in df.columns:
        return df['Close']
    elif 'precio_cierre' in df.columns:
        return df['precio_cierre']
    
    return None

# Recopilar series diarias con datos post-2025
series_para_graficar = []

print("\n🔍 Buscando series diarias con datos posteriores a enero 2025...")
print("-"*60)

for fuente_nombre, fuente_datos in all_data.items():
    if isinstance(fuente_datos, dict):
        for serie_nombre, df in fuente_datos.items():
            if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
                # Normalizar fechas
                df_copy = df.copy()
                df_copy['fecha'] = df_copy['fecha'].apply(normalizar_fecha)
                
                # Verificar si hay datos después de 2025
                df_2025 = df_copy[pd.to_datetime(df_copy['fecha']) > fecha_corte]
                
                if not df_2025.empty:
                    # Obtener valor de cierre
                    valores = obtener_valor_cierre(df_2025, fuente_nombre)
                    
                    if valores is not None:
                        # Verificar que sea una serie diaria (más de 20 puntos por mes en promedio)
                        dias_rango = (df_2025['fecha'].max() - df_2025['fecha'].min()).days
                        if dias_rango > 0:
                            puntos_por_dia = len(df_2025) / dias_rango
                            
                            # Considerar diaria si tiene más de 0.5 puntos por día (excluyendo fines de semana)
                            if puntos_por_dia > 0.5:
                                series_para_graficar.append({
                                    'fuente': fuente_nombre,
                                    'serie': serie_nombre,
                                    'datos': df_2025,
                                    'valores': valores,
                                    'fecha_min': df_2025['fecha'].min(),
                                    'fecha_max': df_2025['fecha'].max(),
                                    'num_puntos': len(df_2025),
                                    'ultimo_valor': valores.iloc[-1] if len(valores) > 0 else None
                                })
                                print(f"✅ [{fuente_nombre}] {serie_nombre}: {len(df_2025)} puntos desde {df_2025['fecha'].min().strftime('%Y-%m-%d')}")

print(f"\n📈 Total de series diarias encontradas: {len(series_para_graficar)}")

# Crear visualización si hay series
if series_para_graficar:
    # Agrupar por fuente para organizar subplots
    fuentes_unicas = list(set([s['fuente'] for s in series_para_graficar]))
    
    # Determinar layout de subplots
    n_fuentes = len(fuentes_unicas)
    n_cols = 2
    n_rows = (n_fuentes + n_cols - 1) // n_cols
    
    # Crear subplots
    fig = make_subplots(
        rows=n_rows, 
        cols=n_cols,
        subplot_titles=[f"{fuente}" for fuente in fuentes_unicas[:n_rows*n_cols]],
        vertical_spacing=0.1,
        horizontal_spacing=0.15
    )
    
    # Colores por fuente
    color_map = {
        'FRED': '#1f77b4',
        'INEGI': '#2ca02c',
        'banxico': '#d62728',
        'world': '#9467bd',
        'ahmsa': '#ff7f0e',
        'trading': '#8c564b',
        'YahooFinance': '#e377c2',
        'LME': '#7f7f7f',
        'RawMaterials': '#17becf'
    }
    
    # Agregar series al gráfico
    for idx, fuente in enumerate(fuentes_unicas[:n_rows*n_cols]):
        row = idx // n_cols + 1
        col = idx % n_cols + 1
        
        # Filtrar series de esta fuente
        series_fuente = [s for s in series_para_graficar if s['fuente'] == fuente]
        
        # Agregar cada serie
        for i, serie_info in enumerate(series_fuente[:10]):  # Máximo 10 series por subplot
            df_plot = serie_info['datos']
            valores_plot = serie_info['valores']
            
            # Resetear índice si es necesario
            if not isinstance(valores_plot.index, pd.RangeIndex):
                valores_plot = valores_plot.reset_index(drop=True)
            
            fig.add_trace(
                go.Scatter(
                    x=df_plot['fecha'],
                    y=valores_plot,
                    name=serie_info['serie'][:30],  # Limitar longitud del nombre
                    mode='lines',
                    line=dict(width=1.5, color=color_map.get(fuente, 'black')),
                    opacity=0.8 - (i * 0.05),  # Reducir opacidad para series adicionales
                    showlegend=(i < 3),  # Solo mostrar leyenda para las primeras 3
                    hovertemplate=f"<b>{serie_info['serie']}</b><br>Fecha: %{{x|%Y-%m-%d}}<br>Valor: %{{y:.2f}}<extra></extra>"
                ),
                row=row, col=col
            )
        
        # Actualizar ejes
        fig.update_xaxes(
            title_text="",
            tickformat="%Y-%m",
            tickangle=45,
            row=row, col=col
        )
        fig.update_yaxes(
            title_text="Valor",
            row=row, col=col
        )
    
    # Actualizar layout general
    fig.update_layout(
        height=300 * n_rows,
        title_text="📊 Series Temporales Diarias con Datos Post-2025 (Valores de Cierre)",
        title_font_size=16,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02,
            font=dict(size=10)
        ),
        hovermode='x unified'
    )
    
    # Mostrar figura
    fig.show()
    
    # Resumen estadístico
    print("\n📊 RESUMEN ESTADÍSTICO")
    print("-"*60)
    
    # Agrupar por fuente para estadísticas
    estadisticas_por_fuente = {}
    for serie in series_para_graficar:
        fuente = serie['fuente']
        if fuente not in estadisticas_por_fuente:
            estadisticas_por_fuente[fuente] = {
                'num_series': 0,
                'total_puntos': 0,
                'series': []
            }
        estadisticas_por_fuente[fuente]['num_series'] += 1
        estadisticas_por_fuente[fuente]['total_puntos'] += serie['num_puntos']
        estadisticas_por_fuente[fuente]['series'].append(serie['serie'])
    
    # Mostrar estadísticas
    for fuente, stats in sorted(estadisticas_por_fuente.items()):
        print(f"\n📌 {fuente}:")
        print(f"   • Series: {stats['num_series']}")
        print(f"   • Total puntos en 2025: {stats['total_puntos']:,}")
        print(f"   • Series incluidas: {', '.join(stats['series'][:5])}")
        if len(stats['series']) > 5:
            print(f"     ... y {len(stats['series']) - 5} más")
    
    # TOP 5 series con más datos
    print("\n🏆 TOP 5 SERIES CON MÁS DATOS EN 2025:")
    print("-"*40)
    top_series = sorted(series_para_graficar, key=lambda x: x['num_puntos'], reverse=True)[:5]
    for i, serie in enumerate(top_series, 1):
        print(f"{i}. [{serie['fuente']}] {serie['serie']}: {serie['num_puntos']} puntos")
        if serie['ultimo_valor'] is not None:
            print(f"   Último valor: {serie['ultimo_valor']:.2f}")
    
else:
    print("\n⚠️ No se encontraron series diarias con datos posteriores a enero 2025")
    print("   Verificar que la ingesta de datos se haya ejecutado correctamente")

print("\n" + "="*80)


## 🔗 Análisis de Correlaciones

Analizamos las correlaciones entre las diferentes series temporales y el precio de la varilla.


In [ ]:
# FUNCIÓN HELPER: Manejo correcto de fechas con timezone y normalización
def safe_to_datetime(fecha_series, normalize=True):
    """
    Convierte de manera segura una serie de fechas a datetime, manejando timezones.
    
    Args:
        fecha_series: Serie de pandas con fechas (puede tener timezone o no)
        normalize: Si True, elimina la hora y deja solo fecha (año-mes-día)
    
    Returns:
        Serie de pandas con fechas sin timezone (timezone-naive) y normalizadas
    """
    try:
        # Intento 1: Conversión directa
        result = pd.to_datetime(fecha_series)
    except (ValueError, TypeError):
        try:
            # Intento 2: Con UTC=True y luego eliminar timezone
            result = pd.to_datetime(fecha_series, utc=True)
            result = result.dt.tz_localize(None)
        except:
            # Intento 3: Convertir elemento por elemento
            result = fecha_series.apply(lambda x: 
                pd.to_datetime(x).tz_localize(None) if hasattr(pd.to_datetime(x), 'tz_localize') 
                else pd.to_datetime(x))
    
    # Verificación final: eliminar timezone si aún existe
    if hasattr(result.dtype, 'tz') and result.dt.tz is not None:
        result = result.dt.tz_localize(None)
    
    # NORMALIZAR: Eliminar horas, minutos, segundos (dejar solo fecha)
    if normalize:
        result = result.dt.normalize()  # Esto pone la hora en 00:00:00
    
    return result

print("✅ Función helper para manejo de fechas con timezone y normalización definida")


In [ ]:
# DIAGNÓSTICO: Visualizar el proceso de JOIN de variables diarias
print("="*80)
print("🔍 DIAGNÓSTICO: PROCESO DE JOIN DE VARIABLES DIARIAS")
print("="*80)

# Fecha de corte para verificar si las series están actualizadas
fecha_corte = pd.Timestamp('2025-01-01')

# 1. Primero, obtener la variable objetivo
print("\n1️⃣ VARIABLE OBJETIVO:")
print("-"*40)

variable_objetivo_df = None
if 'LME' in all_data and 'steel_rebar' in all_data['LME']:
    df = all_data['LME']['steel_rebar']
    if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
        # Buscar columna de valor
        if 'Close' in df.columns:
            valor_col = 'Close'
        elif 'precio_cierre' in df.columns:
            valor_col = 'precio_cierre'
        elif 'valor' in df.columns:
            valor_col = 'valor'
        else:
            valor_col = df.select_dtypes(include=['float64', 'int64']).columns[0] if len(df.select_dtypes(include=['float64', 'int64']).columns) > 0 else None
        
        if valor_col:
            variable_objetivo_df = df[['fecha', valor_col]].copy()
            variable_objetivo_df.columns = ['fecha', 'precio_varilla_lme']
            
            # Usar la función helper para manejar fechas con timezone Y NORMALIZAR
            variable_objetivo_df['fecha'] = safe_to_datetime(variable_objetivo_df['fecha'], normalize=True)
            
            variable_objetivo_df.set_index('fecha', inplace=True)
            
            # IMPORTANTE: Después de normalizar, pueden haber duplicados
            # Mantener el promedio de valores duplicados para la misma fecha
            if variable_objetivo_df.index.duplicated().any():
                print(f"   ⚠️ Fechas duplicadas detectadas después de normalizar. Agregando por promedio...")
                variable_objetivo_df = variable_objetivo_df.groupby(level=0).mean()
            
            variable_objetivo_df = variable_objetivo_df.dropna()
            
            print(f"✅ LME steel_rebar encontrada:")
            print(f"   • Registros totales: {len(variable_objetivo_df)}")
            print(f"   • Rango de fechas: {variable_objetivo_df.index.min()} a {variable_objetivo_df.index.max()}")
            print(f"   • Frecuencia aparente: {pd.infer_freq(variable_objetivo_df.index[:100])}")
            print(f"   • Primeros valores:")
            print(variable_objetivo_df.head(3))
            print(f"   • Últimos valores:")
            print(variable_objetivo_df.tail(3))

# 2. Analizar algunas series diarias antes del join
print("\n2️⃣ ANÁLISIS DE SERIES DIARIAS INDIVIDUALES:")
print("-"*40)

# Seleccionar algunas series diarias clave para analizar
series_to_check = [
    ('banxico', 'usd_mxn', 'tipo_cambio'),
    ('YahooFinance', 'SP500', 'sp500'),
    ('LME', 'Cobre', 'cobre_lme'),
    ('FRED', 'dxy_index', 'dxy_index'),
    ('RawMaterials', 'MineralHierro_VALE', 'vale')
]

series_dataframes = {}
for source, series_key, name in series_to_check:
    if source in all_data:
        # Buscar la serie (puede tener fecha añadida)
        df = None
        if series_key in all_data[source]:
            df = all_data[source][series_key]
        else:
            # Buscar con fecha
            matching = [k for k in all_data[source].keys() if k.startswith(series_key) and 'metadata' not in k]
            if matching:
                df = all_data[source][matching[0]]
        
        if df is not None and isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
            # Buscar columna de valor
            valor_col = None
            for col in ['Close', 'precio_cierre', 'valor']:
                if col in df.columns:
                    valor_col = col
                    break
            
            if not valor_col and len(df.columns) > 1:
                # Tomar la primera columna numérica que no sea fecha
                numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
                valor_col = numeric_cols[0] if len(numeric_cols) > 0 else None
            
            if valor_col:
                temp_df = df[['fecha', valor_col]].copy()
                temp_df.columns = ['fecha', name]
                
                # Usar la función helper para manejar fechas con timezone Y NORMALIZAR
                temp_df['fecha'] = safe_to_datetime(temp_df['fecha'], normalize=True)
                
                temp_df.set_index('fecha', inplace=True)
                
                # Manejar duplicados después de normalizar
                if temp_df.index.duplicated().any():
                    num_duplicates = temp_df.index.duplicated().sum()
                    print(f"      ⚠️ {num_duplicates} fechas duplicadas. Agregando por promedio...")
                    temp_df = temp_df.groupby(level=0).mean()
                
                temp_df = temp_df.dropna()
                
                series_dataframes[name] = temp_df
                
                print(f"\n📊 {source} - {name}:")
                print(f"   • Registros: {len(temp_df)}")
                print(f"   • Rango: {temp_df.index.min()} a {temp_df.index.max()}")
                print(f"   • Actualizada: {'✅ SÍ' if temp_df.index.max() > fecha_corte else '❌ NO'}")
                print(f"   • Muestra de valores: {temp_df[name].head(3).values}")

# 3. Proceso de JOIN
print("\n3️⃣ PROCESO DE JOIN (OUTER):")
print("-"*40)

if variable_objetivo_df is not None and len(series_dataframes) > 0:
    # Empezar con la variable objetivo
    consolidated = variable_objetivo_df.copy()
    print(f"\nIniciando con variable objetivo: {consolidated.shape}")
    
    # Hacer join con cada serie
    for i, (name, df) in enumerate(series_dataframes.items(), 1):
        print(f"\n{i}. JOIN con {name}:")
        print(f"   • DataFrame actual: {consolidated.shape}")
        print(f"   • Serie a unir: {df.shape}")
        
        # Hacer el join
        consolidated = consolidated.join(df, how='outer')
        
        print(f"   • Resultado después del join: {consolidated.shape}")
        print(f"   • NaN introducidos: {consolidated.isna().sum().sum() - consolidated.iloc[:, :-1].isna().sum().sum()}")
    
    print("\n4️⃣ ESTADO FINAL DEL DATAFRAME CONSOLIDADO:")
    print("-"*40)
    print(f"• Forma final: {consolidated.shape}")
    print(f"• Rango de fechas: {consolidated.index.min()} a {consolidated.index.max()}")
    print(f"• Total de NaN: {consolidated.isna().sum().sum()}")
    print(f"• ¿Índice monótono?: {consolidated.index.is_monotonic_increasing}")
    print(f"• ¿Fechas duplicadas?: {consolidated.index.duplicated().any()}")
    
    # Análisis de NaN por columna
    print("\n5️⃣ ANÁLISIS DE NaN POR COLUMNA:")
    print("-"*40)
    nan_analysis = pd.DataFrame({
        'columna': consolidated.columns,
        'total_valores': len(consolidated),
        'valores_no_nan': consolidated.count(),
        'valores_nan': consolidated.isna().sum(),
        'porcentaje_nan': (consolidated.isna().sum() / len(consolidated) * 100).round(1)
    })
    print(nan_analysis)
    
    # Mostrar algunas filas del DataFrame consolidado
    print("\n6️⃣ MUESTRA DEL DATAFRAME CONSOLIDADO:")
    print("-"*40)
    print("Primeras 5 filas:")
    print(consolidated.head())
    print("\nÚltimas 5 filas:")
    print(consolidated.tail())
    
    # Análisis de gaps en las fechas
    print("\n7️⃣ ANÁLISIS DE GAPS EN FECHAS:")
    print("-"*40)
    date_diffs = consolidated.index.to_series().diff().dropna()
    print(f"• Diferencia mínima entre fechas: {date_diffs.min()}")
    print(f"• Diferencia máxima entre fechas: {date_diffs.max()}")
    print(f"• Diferencia promedio: {date_diffs.mean()}")
    print(f"• Mediana de diferencias: {date_diffs.median()}")
    
    # Distribución de gaps
    gaps_distribution = date_diffs.value_counts().head(10)
    print("\nDistribución de gaps (top 10):")
    for gap, count in gaps_distribution.items():
        print(f"   • {gap.days} días: {count} veces")

print("\n" + "="*80)
print("💡 OBSERVACIONES CLAVE:")
print("="*80)
print("""
1. El JOIN 'outer' preserva TODAS las fechas de TODAS las series
2. Esto introduce muchos NaN donde una serie no tiene datos en fechas específicas
3. Series con diferentes rangos de fechas causan NaN en los extremos
4. Series diarias vs mensuales causan muchos NaN en días intermedios
5. El índice puede quedar desordenado después de múltiples joins

RECOMENDACIÓN: Usar solo series diarias y actualizadas para minimizar NaN
""")


In [ ]:
# Crear matriz de correlaciones - ACTUALIZADA CON DATOS REALES
def create_correlation_matrix(all_data: Dict) -> pd.DataFrame:
    """Crea una matriz de correlación entre todas las series temporales"""
    
    # Crear DataFrame consolidado con todas las series
    consolidated_df = pd.DataFrame()
    
    # Variable objetivo: precio de varilla (LME steel_rebar)
    print("🎯 Buscando variable objetivo (precio de varilla)...")
    
    # Intentar con LME steel_rebar primero
    if 'LME' in all_data and 'steel_rebar' in all_data['LME']:
        df = all_data['LME']['steel_rebar']
        if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
            # Buscar columna de valor (Close, precio_cierre, valor)
            valor_col = None
            if 'Close' in df.columns:
                valor_col = 'Close'
            elif 'precio_cierre' in df.columns:
                valor_col = 'precio_cierre'
            elif 'valor' in df.columns:
                valor_col = 'valor'
            
            if valor_col:
                df_pivot = df[['fecha', valor_col]].copy()
                df_pivot.columns = ['fecha', 'precio_varilla_lme']
                
                # Usar función helper para normalizar fechas
                df_pivot['fecha'] = safe_to_datetime(df_pivot['fecha'], normalize=True)
                
                df_pivot.set_index('fecha', inplace=True)
                
                # Manejar duplicados después de normalizar
                if df_pivot.index.duplicated().any():
                    df_pivot = df_pivot.groupby(level=0).mean()
                consolidated_df = df_pivot
                print(f"   ✅ Variable objetivo encontrada: LME steel_rebar ({len(df_pivot)} puntos)")
    
    # Si no encontramos LME, intentar con AHMSA
    if consolidated_df.empty and 'ahmsa' in all_data:
        # Buscar la clave correcta para AHMSA
        ahmsa_keys = [k for k in all_data['ahmsa'].keys() if 'ahmsa' in k.lower() and 'metadata' not in k]
        if ahmsa_keys:
            df = all_data['ahmsa'][ahmsa_keys[0]]
            if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
                valor_col = None
                if 'Close' in df.columns:
                    valor_col = 'Close'
                elif 'precio_cierre' in df.columns:
                    valor_col = 'precio_cierre'
                elif 'valor' in df.columns:
                    valor_col = 'valor'
                
                if valor_col:
                    df_pivot = df[['fecha', valor_col]].copy()
                    df_pivot.columns = ['fecha', 'precio_varilla_ahmsa']
                    
                    # Usar función helper para normalizar fechas
                    df_pivot['fecha'] = safe_to_datetime(df_pivot['fecha'], normalize=True)
                    
                    df_pivot.set_index('fecha', inplace=True)
                    
                    # Manejar duplicados después de normalizar
                    if df_pivot.index.duplicated().any():
                        df_pivot = df_pivot.groupby(level=0).mean()
                    consolidated_df = df_pivot
                    print(f"   ✅ Variable objetivo alternativa: AHMSA ({len(df_pivot)} puntos)")
    
    if consolidated_df.empty:
        print("   ⚠️ No se encontró variable objetivo")
        return pd.DataFrame(), pd.DataFrame()
    
    print("\n📊 Agregando series explicativas...")
    
    # Agregar otras series importantes con las claves correctas
    series_to_add = [
        # Banxico - usar las claves con fecha
        ('banxico', 'usd_mxn', 'tipo_cambio', ['Close', 'valor']),
        ('banxico', 'tiie_28', 'tiie_28', ['Close', 'valor']),
        ('banxico', 'interest_rate', 'tasa_interes', ['Close', 'valor']),
        ('banxico', 'udis', 'udis', ['Close', 'valor']),
        
        # FRED
        ('FRED', 'federal_funds_rate', 'tasa_fed', ['valor', 'Close']),
        ('FRED', 'ProduccionIndustrial', 'prod_industrial_us', ['valor', 'Close']),
        ('FRED', 'steel_production', 'produccion_acero_us', ['valor', 'Close']),
        ('FRED', 'ppi_metals', 'ppi_metales', ['valor', 'Close']),
        ('FRED', 'dxy_index', 'dxy_index', ['valor', 'Close']),
        ('FRED', 'natural_gas', 'gas_natural', ['valor', 'Close']),
        
        # LME - Metales
        ('LME', 'Cobre', 'cobre_lme', ['Close', 'precio_cierre', 'valor']),
        ('LME', 'Aluminio', 'aluminio_lme', ['Close', 'precio_cierre', 'valor']),
        ('LME', 'Zinc', 'zinc_lme', ['Close', 'precio_cierre', 'valor']),
        ('LME', 'iron_ore', 'mineral_hierro_lme', ['Close', 'precio_cierre', 'valor']),
        ('LME', 'coking_coal', 'carbon_coque_lme', ['Close', 'precio_cierre', 'valor']),
        
        # INEGI
        ('INEGI', 'inpc_general', 'inflacion_mx', ['valor', 'Close']),
        ('INEGI', 'ProduccionConstruccion', 'construccion_mx', ['valor', 'Close']),
        ('INEGI', 'produccion_metalurgica', 'prod_metalurgica_mx', ['valor', 'Close']),
        ('INEGI', 'inpp_construccion', 'inpp_construccion', ['valor', 'Close']),
        
        # Raw Materials
        ('RawMaterials', 'MineralHierro_VALE', 'vale_mineral_hierro', ['Close', 'precio_cierre', 'valor']),
        ('RawMaterials', 'MineralHierro_RIO', 'rio_mineral_hierro', ['Close', 'precio_cierre', 'valor']),
        ('RawMaterials', 'MineralHierro_BHP', 'bhp_mineral_hierro', ['Close', 'precio_cierre', 'valor']),
        ('RawMaterials', 'ETF_Acero_SLX', 'etf_acero_slx', ['Close', 'precio_cierre', 'valor']),
        ('RawMaterials', 'CarbonCoque_TECK', 'carbon_coque_teck', ['Close', 'precio_cierre', 'valor']),
        
        # Yahoo Finance
        ('YahooFinance', 'SP500', 'sp500', ['Close', 'precio_cierre', 'valor']),
        ('YahooFinance', 'Petroleo_WTI', 'petroleo_wti', ['Close', 'precio_cierre', 'valor']),
        ('YahooFinance', 'Petroleo_Brent', 'petroleo_brent', ['Close', 'precio_cierre', 'valor']),
        ('YahooFinance', 'commodities_etf', 'commodities_etf', ['Close', 'precio_cierre', 'valor']),
        ('YahooFinance', 'materials_etf', 'materials_etf', ['Close', 'precio_cierre', 'valor']),
        ('YahooFinance', 'VIX_Volatilidad', 'vix', ['Close', 'precio_cierre', 'valor']),
        ('YahooFinance', 'treasury_10y', 'bonos_10y', ['Close', 'precio_cierre', 'valor']),
    ]
    
    # También buscar las series de AHMSA con fecha
    if 'ahmsa' in all_data:
        for key in all_data['ahmsa'].keys():
            if 'metadata' not in key:
                # Agregar las series de AHMSA que no sean la principal
                if 'nucor' in key.lower():
                    series_to_add.append(('ahmsa', key, 'nucor', ['Close', 'precio_cierre', 'valor']))
                elif 'arcelormittal' in key.lower():
                    series_to_add.append(('ahmsa', key, 'arcelormittal', ['Close', 'precio_cierre', 'valor']))
                elif 'ternium' in key.lower():
                    series_to_add.append(('ahmsa', key, 'ternium', ['Close', 'precio_cierre', 'valor']))
                elif 'steel_etf' in key.lower():
                    series_to_add.append(('ahmsa', key, 'steel_etf_ahmsa', ['Close', 'precio_cierre', 'valor']))
    
    # Procesar cada serie
    series_added = []
    for source, series, name, value_cols in series_to_add:
        if source in all_data:
            # Si la serie no tiene fecha en el nombre, buscar con fecha
            if series in all_data[source]:
                df = all_data[source][series]
            else:
                # Buscar la serie con fecha añadida
                matching_keys = [k for k in all_data[source].keys() 
                               if k.startswith(series) and 'metadata' not in k]
                if matching_keys:
                    df = all_data[source][matching_keys[0]]
                else:
                    continue
            
            if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
                # Buscar columna de valor
                valor_col = None
                for col in value_cols:
                    if col in df.columns:
                        valor_col = col
                        break
                
                if valor_col:
                    df_temp = df[['fecha', valor_col]].copy()
                    df_temp.columns = ['fecha', name]
                    
                    # Usar función helper para normalizar fechas
                    df_temp['fecha'] = safe_to_datetime(df_temp['fecha'], normalize=True)
                    
                    df_temp.set_index('fecha', inplace=True)
                    
                    # Manejar duplicados después de normalizar
                    if df_temp.index.duplicated().any():
                        df_temp = df_temp.groupby(level=0).mean()
                    
                    # Eliminar valores NaN
                    df_temp = df_temp.dropna()
                    
                    if len(df_temp) > 0:
                        # Unir con el DataFrame consolidado
                        consolidated_df = consolidated_df.join(df_temp, how='outer')
                        series_added.append(name)
    
    print(f"   ✅ Series agregadas: {len(series_added)}")
    
    # Calcular correlaciones solo con datos completos
    if not consolidated_df.empty:
        print("\n📈 Calculando matriz de correlación...")
        
        # IMPORTANTE: Ordenar el índice antes de resamplear
        print("   Ordenando índice de fechas...")
        consolidated_df = consolidated_df.sort_index()
        
        # Verificar si el índice es monótono
        if not consolidated_df.index.is_monotonic_increasing:
            print("   ⚠️ Índice no monótono detectado, eliminando duplicados...")
            # Eliminar fechas duplicadas (mantener el primer valor)
            consolidated_df = consolidated_df[~consolidated_df.index.duplicated(keep='first')]
            consolidated_df = consolidated_df.sort_index()
        
        # Solo resamplear si tenemos series con diferentes frecuencias
        # Para series diarias, esto no es necesario
        print("   Verificando necesidad de resampling...")
        date_diffs = consolidated_df.index.to_series().diff().dropna()
        if date_diffs.min() > pd.Timedelta(days=1):
            # Hay gaps mayores a 1 día, probablemente series mensuales mezcladas
            print("   Resampling series a frecuencia diaria...")
            consolidated_df = consolidated_df.resample('D').ffill()
        else:
            print("   No es necesario resamplear (todas las series son diarias)")
        
        # Ahora eliminar columnas con demasiados NaN
        print("   Eliminando columnas con >80% NaN...")
        nan_threshold = 0.8
        for col in consolidated_df.columns:
            nan_ratio = consolidated_df[col].isna().sum() / len(consolidated_df)
            if nan_ratio > nan_threshold:
                print(f"      Eliminando {col}: {nan_ratio:.1%} NaN")
                consolidated_df = consolidated_df.drop(columns=[col])
        
        # Eliminar filas con muchos NaN
        threshold = max(2, len(consolidated_df.columns) * 0.3)  # Al menos 30% de datos presentes
        before_rows = len(consolidated_df)
        consolidated_df = consolidated_df.dropna(thresh=threshold)
        after_rows = len(consolidated_df)
        
        print(f"   Filas con datos suficientes: {after_rows}/{before_rows}")
        print(f"   Variables en análisis: {len(consolidated_df.columns)}")
        
        # Para las correlaciones, usar solo pares de datos completos
        if len(consolidated_df) > 30:  # Necesitamos al menos 30 observaciones
            # Calcular correlación usando pairwise (ignora NaN en pares)
            correlation_matrix = consolidated_df.corr(method='pearson', min_periods=30)
            
            # Contar correlaciones válidas
            valid_corrs = (~correlation_matrix.isna()).sum().sum()
            total_corrs = len(correlation_matrix.columns) ** 2
            
            print(f"   ✅ Matriz de correlación calculada: {correlation_matrix.shape}")
            print(f"   Correlaciones válidas: {valid_corrs}/{total_corrs} ({valid_corrs/total_corrs:.1%})")
            
            # Mostrar variables con más correlaciones válidas
            valid_counts = (~correlation_matrix.isna()).sum()
            top_vars = valid_counts.nlargest(10)
            print("\n   📊 Variables con más correlaciones válidas:")
            for var, count in top_vars.items():
                print(f"      {var}: {count}/{len(correlation_matrix.columns)-1} correlaciones")
            
            return correlation_matrix, consolidated_df
        else:
            print(f"   ⚠️ Datos insuficientes para correlación ({len(consolidated_df)} filas)")
            return pd.DataFrame(), consolidated_df
    else:
        print("   ⚠️ DataFrame consolidado vacío")
        return pd.DataFrame(), pd.DataFrame()

# Calcular correlaciones
print("="*80)
print("🔍 ANÁLISIS DE CORRELACIONES")
print("="*80)
corr_matrix, consolidated_df = create_correlation_matrix(all_data)

if not corr_matrix.empty:
    # Buscar la columna de precio de varilla (puede ser precio_varilla_lme o precio_varilla_ahmsa)
    target_col = None
    if 'precio_varilla_lme' in corr_matrix.columns:
        target_col = 'precio_varilla_lme'
    elif 'precio_varilla_ahmsa' in corr_matrix.columns:
        target_col = 'precio_varilla_ahmsa'
    
    if target_col:
        # Mostrar correlaciones con precio de varilla
        print("\n🔗 CORRELACIONES CON PRECIO DE VARILLA")
        print("="*60)
        print(f"Variable objetivo: {target_col}")
        print("-"*60)
        
        correlations = corr_matrix[target_col].sort_values(ascending=False)
        
        # Separar por categorías de fuerza
        muy_fuerte = []
        fuerte = []
        moderada = []
        debil = []
        
        for var, corr in correlations.items():
            if var != target_col and not pd.isna(corr):
                abs_corr = abs(corr)
                item = (var, corr)
                
                if abs_corr > 0.8:
                    muy_fuerte.append(item)
                elif abs_corr > 0.6:
                    fuerte.append(item)
                elif abs_corr > 0.4:
                    moderada.append(item)
                else:
                    debil.append(item)
        
        # Mostrar por categorías
        if muy_fuerte:
            print("\n🔴 CORRELACIÓN MUY FUERTE (|r| > 0.8):")
            for var, corr in muy_fuerte:
                direction = "↑" if corr > 0 else "↓"
                print(f"   {direction} {var:25s}: {corr:+.3f}")
        
        if fuerte:
            print("\n🟠 CORRELACIÓN FUERTE (0.6 < |r| ≤ 0.8):")
            for var, corr in fuerte:
                direction = "↑" if corr > 0 else "↓"
                print(f"   {direction} {var:25s}: {corr:+.3f}")
        
        if moderada:
            print("\n🟡 CORRELACIÓN MODERADA (0.4 < |r| ≤ 0.6):")
            for var, corr in moderada[:10]:  # Mostrar solo las 10 primeras
                direction = "↑" if corr > 0 else "↓"
                print(f"   {direction} {var:25s}: {corr:+.3f}")
            if len(moderada) > 10:
                print(f"   ... y {len(moderada)-10} más")
        
        if debil:
            print(f"\n🟢 CORRELACIÓN DÉBIL (|r| ≤ 0.4): {len(debil)} variables")
            # Mostrar solo las 5 más fuertes de las débiles
            for var, corr in sorted(debil, key=lambda x: abs(x[1]), reverse=True)[:5]:
                direction = "↑" if corr > 0 else "↓"
                print(f"   {direction} {var:25s}: {corr:+.3f}")
        
        print("\n" + "="*60)
    
    # Crear heatmap de correlaciones
    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale='RdBu',
        zmid=0,
        text=corr_matrix.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 10},
        colorbar=dict(title="Correlación")
    ))
    
    fig.update_layout(
        title="Matriz de Correlación entre Variables",
        width=800,
        height=800,
        xaxis_title="Variables",
        yaxis_title="Variables"
    )
    
    fig.show()
    
    print("\n📊 Matriz de correlación generada")
else:
    print("⚠️ No se pudo calcular la matriz de correlación - datos insuficientes")


## 🔗 Correlación con Commodities Mensuales

Análisis específico de correlación entre el precio de la varilla y los commodities mensuales del World Bank.


In [ ]:
# DIAGNÓSTICO: Visualizar el proceso de JOIN de variables MENSUALES actualizadas
print("="*80)
print("🔍 DIAGNÓSTICO: PROCESO DE JOIN DE VARIABLES MENSUALES ACTUALIZADAS")
print("="*80)

# Fecha de corte para verificar si las series están actualizadas
fecha_corte = pd.Timestamp('2025-01-01')

# 1. Primero, identificar todas las series mensuales actualizadas
print("\n1️⃣ IDENTIFICACIÓN DE SERIES MENSUALES ACTUALIZADAS:")
print("-"*40)

series_mensuales_actualizadas = {}

# FRED - Series mensuales
fred_mensuales = [
    ('federal_funds_rate', 'tasa_fed'),
    ('steel_production', 'produccion_acero_us'),
    ('ppi_metals', 'ppi_metales'),
    ('iron_steel_scrap', 'chatarra_acero'),
    ('GastoConstruccion', 'gasto_construccion_us'),
    ('ProduccionIndustrial', 'produccion_industrial_us')
]

if 'FRED' in all_data:
    for series_key, nombre in fred_mensuales:
        # Buscar la serie
        df = None
        if series_key in all_data['FRED']:
            df = all_data['FRED'][series_key]
        else:
            # Buscar con fecha
            matching = [k for k in all_data['FRED'].keys() if k.startswith(series_key) and 'metadata' not in k]
            if matching:
                df = all_data['FRED'][matching[0]]
        
        if df is not None and isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
            # Usar safe_to_datetime para manejar fechas
            df['fecha'] = safe_to_datetime(df['fecha'])
            fecha_max = df['fecha'].max()
            
            if fecha_max >= fecha_corte:
                # Buscar columna de valor
                valor_col = None
                for col in ['valor', 'value', 'Value']:
                    if col in df.columns:
                        valor_col = col
                        break
                
                if not valor_col and len(df.columns) > 1:
                    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
                    valor_col = numeric_cols[0] if len(numeric_cols) > 0 else None
                
                if valor_col:
                    temp_df = df[['fecha', valor_col]].copy()
                    temp_df.columns = ['fecha', nombre]
                    temp_df['fecha'] = safe_to_datetime(temp_df['fecha'])
                    temp_df.set_index('fecha', inplace=True)
                    
                    # Manejar duplicados
                    if temp_df.index.duplicated().any():
                        temp_df = temp_df.groupby(level=0).mean()
                    
                    temp_df = temp_df.dropna()
                    series_mensuales_actualizadas[nombre] = temp_df
                    
                    print(f"✅ FRED - {nombre}:")
                    print(f"   • Registros: {len(temp_df)}")
                    print(f"   • Rango: {temp_df.index.min()} a {temp_df.index.max()}")

# INEGI - Series mensuales actualizadas
inegi_mensuales = [
    ('ProduccionConstruccion', 'produccion_construccion_mx'),
    ('produccion_metalurgica', 'produccion_metalurgica_mx')
]

if 'INEGI' in all_data:
    for series_key, nombre in inegi_mensuales:
        # Buscar la serie
        df = None
        if series_key in all_data['INEGI']:
            df = all_data['INEGI'][series_key]
        else:
            matching = [k for k in all_data['INEGI'].keys() if k.startswith(series_key) and 'metadata' not in k]
            if matching:
                df = all_data['INEGI'][matching[0]]
        
        if df is not None and isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
            df['fecha'] = safe_to_datetime(df['fecha'])
            fecha_max = df['fecha'].max()
            
            if fecha_max >= pd.Timestamp('2025-07-01'):  # INEGI tiene datos hasta julio 2025
                valor_col = None
                for col in ['valor', 'value']:
                    if col in df.columns:
                        valor_col = col
                        break
                
                if not valor_col and len(df.columns) > 1:
                    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
                    valor_col = numeric_cols[0] if len(numeric_cols) > 0 else None
                
                if valor_col:
                    temp_df = df[['fecha', valor_col]].copy()
                    temp_df.columns = ['fecha', nombre]
                    temp_df['fecha'] = safe_to_datetime(temp_df['fecha'])
                    temp_df.set_index('fecha', inplace=True)
                    
                    if temp_df.index.duplicated().any():
                        temp_df = temp_df.groupby(level=0).mean()
                    
                    temp_df = temp_df.dropna()
                    series_mensuales_actualizadas[nombre] = temp_df
                    
                    print(f"✅ INEGI - {nombre}:")
                    print(f"   • Registros: {len(temp_df)}")
                    print(f"   • Rango: {temp_df.index.min()} a {temp_df.index.max()}")

# Banxico - Inflación mensual
if 'banxico' in all_data:
    inflation_keys = [k for k in all_data['banxico'].keys() if 'inflation_monthly' in k and 'metadata' not in k]
    if inflation_keys:
        df = all_data['banxico'][inflation_keys[0]]
        if isinstance(df, pd.DataFrame) and 'fecha' in df.columns:
            df['fecha'] = safe_to_datetime(df['fecha'])
            fecha_max = df['fecha'].max()
            
            if fecha_max >= fecha_corte:
                valor_col = None
                for col in ['valor', 'value', 'inflation_rate']:
                    if col in df.columns:
                        valor_col = col
                        break
                
                if not valor_col and len(df.columns) > 1:
                    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
                    valor_col = numeric_cols[0] if len(numeric_cols) > 0 else None
                
                if valor_col:
                    temp_df = df[['fecha', valor_col]].copy()
                    temp_df.columns = ['fecha', 'inflacion_mx']
                    temp_df['fecha'] = safe_to_datetime(temp_df['fecha'])
                    temp_df.set_index('fecha', inplace=True)
                    
                    if temp_df.index.duplicated().any():
                        temp_df = temp_df.groupby(level=0).mean()
                    
                    temp_df = temp_df.dropna()
                    series_mensuales_actualizadas['inflacion_mx'] = temp_df
                    
                    print(f"✅ Banxico - inflacion_mx:")
                    print(f"   • Registros: {len(temp_df)}")
                    print(f"   • Rango: {temp_df.index.min()} a {temp_df.index.max()}")

print(f"\n📊 Total de series mensuales actualizadas: {len(series_mensuales_actualizadas)}")

# 2. Proceso de JOIN de series mensuales
print("\n2️⃣ PROCESO DE JOIN DE SERIES MENSUALES:")
print("-"*40)

if len(series_mensuales_actualizadas) > 0:
    # Empezar con la primera serie
    first_key = list(series_mensuales_actualizadas.keys())[0]
    consolidated_monthly = series_mensuales_actualizadas[first_key].copy()
    print(f"\nIniciando con: {first_key} - Shape: {consolidated_monthly.shape}")
    
    # Hacer join con las demás series
    for i, (name, df) in enumerate(list(series_mensuales_actualizadas.items())[1:], 1):
        print(f"\n{i}. JOIN con {name}:")
        print(f"   • DataFrame actual: {consolidated_monthly.shape}")
        print(f"   • Serie a unir: {df.shape}")
        
        # Hacer el join
        consolidated_monthly = consolidated_monthly.join(df, how='outer')
        
        print(f"   • Resultado después del join: {consolidated_monthly.shape}")
        print(f"   • NaN introducidos: {consolidated_monthly.isna().sum().sum() - consolidated_monthly.iloc[:, :-1].isna().sum().sum()}")
    
    # 3. Análisis del DataFrame consolidado mensual
    print("\n3️⃣ ESTADO FINAL DEL DATAFRAME MENSUAL CONSOLIDADO:")
    print("-"*40)
    print(f"• Forma final: {consolidated_monthly.shape}")
    print(f"• Rango de fechas: {consolidated_monthly.index.min()} a {consolidated_monthly.index.max()}")
    print(f"• Total de NaN: {consolidated_monthly.isna().sum().sum()}")
    print(f"• Porcentaje de datos completos: {(1 - consolidated_monthly.isna().sum().sum() / (consolidated_monthly.shape[0] * consolidated_monthly.shape[1])) * 100:.1f}%")
    
    # Análisis de NaN por columna
    print("\n4️⃣ ANÁLISIS DE COMPLETITUD POR VARIABLE:")
    print("-"*40)
    nan_analysis_monthly = pd.DataFrame({
        'variable': consolidated_monthly.columns,
        'datos_disponibles': consolidated_monthly.count(),
        'datos_faltantes': consolidated_monthly.isna().sum(),
        'pct_completo': (consolidated_monthly.count() / len(consolidated_monthly) * 100).round(1)
    })
    nan_analysis_monthly = nan_analysis_monthly.sort_values('pct_completo', ascending=False)
    print(nan_analysis_monthly)
    
    # Muestra del DataFrame consolidado
    print("\n5️⃣ MUESTRA DEL DATAFRAME MENSUAL CONSOLIDADO:")
    print("-"*40)
    print("Primeras 5 filas:")
    print(consolidated_monthly.head())
    print("\nÚltimas 5 filas:")
    print(consolidated_monthly.tail())
    
    # 6. Resample a diario para integración con series diarias
    print("\n6️⃣ CONVERSIÓN A FRECUENCIA DIARIA (FORWARD FILL):")
    print("-"*40)
    
    # Resample a diario con forward fill
    consolidated_monthly_daily = consolidated_monthly.resample('D').ffill()
    
    print(f"• Shape original (mensual): {consolidated_monthly.shape}")
    print(f"• Shape después de resample (diario): {consolidated_monthly_daily.shape}")
    print(f"• Rango de fechas diario: {consolidated_monthly_daily.index.min()} a {consolidated_monthly_daily.index.max()}")
    
    # Análisis de propagación
    print("\n7️⃣ ANÁLISIS DE PROPAGACIÓN (Forward Fill):")
    print("-"*40)
    print("Ejemplo de cómo se propagan los valores mensuales a diarios:")
    
    # Tomar una muestra de transición de mes
    sample_date = pd.Timestamp('2025-07-28')
    sample_end = pd.Timestamp('2025-08-05')
    
    if sample_date in consolidated_monthly_daily.index and sample_end in consolidated_monthly_daily.index:
        sample = consolidated_monthly_daily.loc[sample_date:sample_end, consolidated_monthly_daily.columns[:3]]
        print(f"\nMuestra de {sample_date.date()} a {sample_end.date()}:")
        print(sample)
    
    # 8. Correlación con variable objetivo (precio varilla)
    print("\n8️⃣ CORRELACIÓN DE VARIABLES MENSUALES CON PRECIO VARILLA:")
    print("-"*40)
    
    # Obtener precio de varilla
    precio_varilla = None
    if 'LME' in all_data and 'steel_rebar' in all_data['LME']:
        df_varilla = all_data['LME']['steel_rebar']
        if isinstance(df_varilla, pd.DataFrame) and 'fecha' in df_varilla.columns:
            valor_col = 'Close' if 'Close' in df_varilla.columns else df_varilla.select_dtypes(include=['float64']).columns[0]
            precio_varilla = df_varilla[['fecha', valor_col]].copy()
            precio_varilla.columns = ['fecha', 'precio_varilla']
            precio_varilla['fecha'] = safe_to_datetime(precio_varilla['fecha'])
            precio_varilla.set_index('fecha', inplace=True)
            
            if precio_varilla.index.duplicated().any():
                precio_varilla = precio_varilla.groupby(level=0).mean()
    
    if precio_varilla is not None and not consolidated_monthly_daily.empty:
        # Unir con precio de varilla
        combined = precio_varilla.join(consolidated_monthly_daily, how='inner')
        
        # Calcular correlaciones
        correlations = combined.corr()['precio_varilla'].drop('precio_varilla').sort_values(ascending=False)
        
        print(f"\nDatos combinados: {combined.shape[0]} observaciones")
        print("\nCorrelaciones con precio de varilla:")
        for var, corr in correlations.items():
            if abs(corr) >= 0.3:
                emoji = "🔴" if abs(corr) > 0.6 else "🟠" if abs(corr) > 0.4 else "🟡"
                print(f"  {emoji} {var:30s}: {corr:+.3f}")

else:
    print("❌ No se encontraron series mensuales actualizadas")

print("\n" + "="*80)
print("💡 OBSERVACIONES CLAVE SOBRE VARIABLES MENSUALES:")
print("="*80)
print("""
1. Las variables mensuales capturan TENDENCIAS de largo plazo
2. Forward fill propaga el valor mensual hasta el siguiente mes
3. Esto introduce autocorrelación artificial en los datos diarios
4. Para modelos predictivos, considerar:
   - MIDAS (Mixed Data Sampling) para preservar frecuencias originales
   - Usar variables mensuales como features de contexto, no principales
   - Aplicar rezagos apropiados (ej: inflación del mes anterior)
5. Variables mensuales más relevantes suelen ser:
   - Inflación (INPC/INPP)
   - Producción industrial/construcción
   - Indicadores económicos generales
""")
